In [1]:
import tensorflow as tf
import os
import collections
import numpy as np

# TODO(zongheng): read from argparse.
# TODO(zongheng): visualization via tensorboard.
# TODO(zongheng): print some prediction from test set

# Toy dataset.
TRAIN_PATH = "data/train-5rel-10numAttrs-26totalAttrs-100maxTblSize-100buckets-30.csv"
TEST_PATH = "data/test-5rel-10numAttrs-26totalAttrs-100maxTblSize-100buckets-30.csv"
NUM_FIELDS = 38
net = [64]
lr = 5e-4
batch_size = 1
num_steps = 7000

# 1000 examples in training.
# Min train loss < 0.6
TRAIN_PATH = "data/train-5rel-10numAttrs-17totalAttrs-100maxTblSize-100buckets-3000.csv"
TEST_PATH = "data/test-5rel-10numAttrs-17totalAttrs-100maxTblSize-100buckets-3000.csv"
NUM_FIELDS = 29
net = [64]
lr = 3e-3
batch_size = 128
num_steps = 5500


# With gaussian weights.
PATH = "5rel-10numAttrs-24totalAttrs-100maxTblSize-10buckets-300.csv"
NUM_FIELDS = 36
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [128, 32, 64]
lr = 3e-3
lr = 1e-2
batch_size = 50
num_steps = 5500

# With gaussian weights.\
PATH = "5rel-10numAttrs-22totalAttrs-100maxTblSize-10buckets-3000.csv"
NUM_FIELDS = 34
TRAIN_PATH = "data/train-{}".format(PATH)
TEST_PATH = "data/test-{}".format(PATH)
net = [64,32]; lr = 1e-2
net = [64, 16, 64]; lr = 5e-3
net = [64]; lr =  5e-4
batch_size = 500
num_steps = 5500

/Users/zongheng/anaconda/envs/ray-0321/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
def DatasetFromPath(pathname):
    dataset = tf.data.TextLineDataset(pathname)

    def ParseLine(line):
        fields = tf.decode_csv(line, [[0.0]] * NUM_FIELDS)
        # Last number is label.
        return tf.reshape(tf.stack(fields[:-1]), [-1]), fields[-1]

    def MeanCenterBatch(feat, lab):
        feat_mean, feat_var = tf.nn.moments(feat, axes=[0])
        lab_mean, lab_var = tf.nn.moments(lab, axes=[0])
#         feat = (feat - feat_mean) / (tf.sqrt(feat_var) + 1e-6)
#         lab = (lab - lab_mean) / (tf.sqrt(lab_var) + 1e-6)
        
        feat = (feat - feat_mean) / (feat_var + 1e-3)
        lab = (lab - lab_mean) / (lab_var + 1e-3)
        feat = tf.Print(feat, [feat_mean, feat_var], message='feat mean/var: ')
        lab = tf.Print(lab, [lab_mean, lab_var], message='lab mean/var: ')
        return feat, lab
    
    # K-point dataset, for sanity checking.
    K = 2
    dataset = dataset.take(K).map(ParseLine)
    dataset = dataset.shuffle(20000, reshuffle_each_iteration=False) \
        .batch(K) \
        .repeat()

#     dataset = dataset.cache().map(ParseLine)
#     dataset = dataset.shuffle(20000, reshuffle_each_iteration=True) \
#         .repeat() \
#         .batch(batch_size)
# #         .map(MeanCenterBatch) 

    print(dataset.output_types)
    print(dataset.output_shapes)
    return dataset

# Train.
dataset = DatasetFromPath(TRAIN_PATH)
iterator = tf.data.Iterator.from_structure(
    dataset.output_types, dataset.output_shapes)
iterator_init = iterator.make_initializer(dataset)
# Test.
test_dataset = DatasetFromPath(TEST_PATH)
test_iterator_init = iterator.make_initializer(test_dataset)
# Shared.
feature_batch, label_batch = iterator.get_next()

    

# mean, var = tf.nn.moments(feature_batch, axes=[0])
# feature_batch = (feature_batch - mean) / tf.sqrt(var)
# mean, var = tf.nn.moments(label_batch, axes=[0])
# label_batch = (label_batch - mean) / var

(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(33)]), TensorShape([Dimension(None)]))
(tf.float32, tf.float32)
(TensorShape([Dimension(None), Dimension(33)]), TensorShape([Dimension(None)]))


In [3]:
# Graph definition.
for i in range(len(net)):
    dense = tf.layers.dense(
        dense if i > 0 else feature_batch,
        net[i],
#         kernel_initializer=tf.truncated_normal_initializer(stddev=20.0),
#         kernel_initializer=tf.initializers.variance_scaling(scale=10.0),
#         kernel_initializer=tf.initializers.orthogonal(),
#         kernel_initializer=tf.initializers.uniform_unit_scaling(),
#         kernel_initializer=tf.initializers.random_normal(stddev=10),
        kernel_initializer=tf.initializers.random_uniform(-100, 100),
#         kernel_initializer=tf.initializers.zeros(),
#         bias_initializer=tf.initializers.zeros(),
        activation=tf.nn.sigmoid,
#         activation=tf.nn.relu,
#         activation=tf.nn.tanh,
    )
out = tf.layers.dense(
    dense,
    1,
#         kernel_initializer=tf.truncated_normal_initializer(stddev=20.0),
#         kernel_initializer=tf.initializers.variance_scaling(scale=10.0),
#         kernel_initializer=tf.initializers.orthogonal(),
#         kernel_initializer=tf.initializers.uniform_unit_scaling(),
#         kernel_initializer=tf.initializers.random_normal(stddev=10),
        kernel_initializer=tf.initializers.random_uniform(-100, 100),
#         kernel_initializer=tf.initializers.zeros(),
#         bias_initializer=tf.initializers.zeros(),
)


all_vars = tf.trainable_variables()
l1_regularizer = tf.contrib.layers.l1_regularizer(scale=0.05)
regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, all_vars)

# loss = tf.reduce_mean(tf.square(out - label_batch))
loss = tf.reduce_mean(tf.square(out - label_batch)) #+ regularization_penalty

optimizer = tf.train.AdamOptimizer(learning_rate=lr)
global_step = tf.train.get_or_create_global_step()
train_step = optimizer.minimize(
    loss, global_step=global_step)


# For visualization.
tf.summary.scalar('Train Loss', loss)
for v in all_vars:
    tf.summary.histogram(v.name, v)
all_vars_norms = [tf.norm(v) for v in all_vars]
print(len(all_vars))

Instructions for updating:
Use the retry module or similar alternatives.
INFO:tensorflow:Summary name Train Loss is illegal; using Train_Loss instead.
INFO:tensorflow:Summary name dense/kernel:0 is illegal; using dense/kernel_0 instead.
INFO:tensorflow:Summary name dense/bias:0 is illegal; using dense/bias_0 instead.
INFO:tensorflow:Summary name dense_1/kernel:0 is illegal; using dense_1/kernel_0 instead.
INFO:tensorflow:Summary name dense_1/bias:0 is illegal; using dense_1/bias_0 instead.
4


In [4]:
# sess = tf.InteractiveSession()

In [ ]:
# Train.
sess_args = {
#              'checkpoint_dir': os.getcwd(),
#              'save_checkpoint_secs': 60, 
             'save_summaries_secs': 1,
             'log_step_count_steps': 5000,
             'hooks': [tf.train.NanTensorHook(loss),
                       tf.train.StopAtStepHook(last_step=2000000),
#                       tf.train.LoggingTensorHook([global_step, loss], every_n_iter=500)
                      ],
            }

loss_vals = collections.deque(maxlen=40)
i = 0
with tf.train.MonitoredTrainingSession(**sess_args) as sess:
    sess.run(iterator_init)
    while not sess.should_stop():
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     running_test_loss = 0.0
#     while True:
        _, loss_val, global_step_val, true_feat_train, true_labels_train, pred_labels_train = sess.run(
            [train_step, loss, global_step,  feature_batch, label_batch, out])

        if i % 1000 == 0:
#             sess.run(test_iterator_init)
#             test_loss_val, actual_labels, predicted_labels = sess.run([loss, label_batch, out])

#             loss_vals.append(test_loss_val)
#             avg_test_loss = np.mean(loss_vals)
#             print('global_step', global_step_val, ', loss', loss_val, ', avg test loss', avg_test_loss)
            print('global_step', global_step_val, ', loss', loss_val)
            if i % 10000 == 0:
#                 print(' true labels', actual_labels[:20].reshape(-1,))
#                 print(' pred labels', predicted_labels[:20].reshape(-1,))
                print(' true feats train', true_feat_train[:2])
                print(' true labels train', true_labels_train[:10].reshape(-1,))
                print(' pred labels train', pred_labels_train[:10].reshape(-1,))
                print(' all vars norms', sess.run(all_vars_norms))
        i += 1
#         if i >= 10000:
#             break

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
global_step 1 , loss 93054.78
 true feats train [[100.  10.  10.  10. 100.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   1.
    0.   0.   0.   0.  94.]
 [100.  10.  10.  10. 100.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.   0.   0.   1.
    0.   0.   0.   0.  53.]]
 true labels train [1. 7.]
 pred labels train [-306.89664 -295.05658]
 all vars norms [2622.2766, 0.0, 470.3299, 0.0005]
global_step 1001 , loss 84364.09
global_step 2001 , loss 76237.414
global_step 3001 , loss 68611.81
global_step 4001 , loss 61450.703
global_step 5001 , loss 54725.195
global_step 6001 , loss 48404.83
global_step 7001 , loss 42507.293
global_step 8001 , loss 37024.008
global_step 9001 , loss 31930.72
global_step 10001 , loss 27242.3
 true feats train [[100.  10.  